In [1]:
import warnings
warnings.filterwarnings('ignore')

import yfinance as yf
import pandas as pd
import numpy as np
import datetime
import project_functions2 as pf
from sklearn.linear_model import LinearRegression

In [25]:
stock_list = ['AMZN', 'AAPL', 'FB','GOOGL', 'MSFT', 'TSLA', 'RIOT']
stock_objects = {}
for stock in stock_list:
    stock_objects[stock] = yf.Ticker(stock)

In [26]:
drop_list = ['Stock Splits','Day','Days From IPO','5 Day Volume Mean','5 Day High Var','5 Day Low Var',
             '5 Day Close Var','5 Day Low','10 Day Low Var','10 Day Close Var','10 Day High','10 Day Low',
             '10 Day SMt','20 Day Low Mean','20 Day Low','5 Day Stochastic K','20 Day Stochastic K','5 Day RSI']

In [27]:
def trading_sim_LRM(stock_objects, drop_list, split_time, time_shift):
    columns = ['Date', 'Fund Value', 'Cash']
    stock_names = []
    curr_cash = 10000
    curr_shares = {}
    
    #stock_dfs = {}
    five_day_stock_dfs = {}
    one_day_stock_dfs = {}
    for key in stock_objects:
        five_day_stock_dfs[key] = stock_objects[key].history(period='max')
        one_day_stock_dfs[key] = stock_objects[key].history(period='max')
    
    for key in five_day_stock_dfs:
        five_day_stock_dfs[key] = pf.date_time_prep(five_day_stock_dfs[key]) 
        five_day_stock_dfs[key] = pf.rolling_aves(five_day_stock_dfs[key])
        five_day_stock_dfs[key] = pf.future_close_setup(five_day_stock_dfs[key], 5)
        five_day_stock_dfs[key].drop(drop_list, axis=1, inplace=True)
        
    for key in one_day_stock_dfs:
        one_day_stock_dfs[key] = pf.date_time_prep(one_day_stock_dfs[key]) 
        one_day_stock_dfs[key] = pf.rolling_aves(one_day_stock_dfs[key])
        one_day_stock_dfs[key] = pf.future_close_setup(one_day_stock_dfs[key], 1)
        one_day_stock_dfs[key].drop(drop_list, axis=1, inplace=True)
    
    five_day_combine_df = pf.combiner(five_day_stock_dfs)
    one_day_combine_df = pf.combiner(one_day_stock_dfs)
    
    test_dfs = five_day_stock_dfs
    
    for key in test_dfs:
        test_dfs[key] = test_dfs[key].tail(split_time)
        curr_shares[key] = 0
        stock_names.append(key)
    
    columns = columns + stock_names
    cash_df = pd.DataFrame(columns=columns)
    curr_line = [combine_df.index[int(len(combine_df) - (split_time * len(stock_dfs)))], curr_cash, curr_cash] + len(stock_names)*[0]
    cash_df.loc[len(cash_df)] = curr_line
    
    while split_time >= time_shift:
        max_stock = ''
        max_stock_gain = 0
        X_train_5, y_train_5, X_test_5, y_test_5 = pf.multi_stock_train_test_split(five_day_combine_df, split_time, five_day_stock_dfs)
        five_day_model = LinearRegression().fit(X_train_5, y_train_5)
        X_train_1, y_train_1, X_test_1, y_test_1 = pf.multi_stock_train_test_split(one_day_combine_df, split_time, one_day_stock_dfs)
        one_day_model = LinearRegression().fit(X_train_1, y_train_1)
        
        for key in test_dfs:
            X = test_dfs[key].iloc[:,:-1]
            stock_5_day_pred = five_day_model.predict(X.head(1))
            stock_5_day_pred = (float(stock_5_day_pred) - X['Close'][0]) / X['Close'][0]
            stock_1_day_pred = one_day_model.predict(X.head(1))
            stock_1_day_pred = (float(stock_1_day_pred) - X['Close'][0]) / X['Close'][0]
            if stock_5_day_pred > max_stock_gain and stock_1_day_pred > 0:
                max_stock = key
                max_stock_gain = stock_5_day_pred
        
        for key in test_dfs:
            test_dfs[key] = test_dfs[key].iloc[1:]
        day_counter = 1
        
        if max_stock_gain > 0:
            if curr_shares[max_stock] == 0:
                for key in curr_shares:
                    curr_cash += curr_shares[key]*test_dfs[key]['Open'][0]
                    curr_shares[key] = 0
                curr_shares[max_stock] = curr_cash // test_dfs[max_stock]['Open'][0]
                curr_cash -= curr_shares[max_stock]*test_dfs[max_stock]['Open'][0]
            else:
                curr_shares[max_stock] += curr_cash // test_dfs[max_stock]['Open'][0]
                curr_cash -= (curr_cash // test_dfs[max_stock]['Open'][0]) * test_dfs[max_stock]['Open'][0]
        else:
            for key in curr_shares:
                    curr_cash += curr_shares[key]*test_dfs[key]['Open'][0]
                    curr_shares[key] = 0
        
        curr_line = [X_test_5.index[len(stock_names)], curr_cash ,curr_cash] + len(stock_names)*[0]
        cash_df.loc[len(cash_df)] = curr_line
        for key in curr_shares:
            cash_df.iloc[-1, cash_df.columns.get_loc(key)] = curr_shares[key]
            cash_df.iloc[-1, cash_df.columns.get_loc('Fund Value')] += curr_shares[key]*test_dfs[key]['Open'][0]
            
        while day_counter < time_shift:
            X_train_1, y_train_1, X_test_1, y_test_1 = pf.multi_stock_train_test_split(one_day_combine_df, split_time-day_counter, one_day_stock_dfs)
            one_day_model = LinearRegression().fit(X_train_1, y_train_1)
            for key in test_dfs:
                test_dfs[key] = test_dfs[key].iloc[1:]
            day_counter += 1
            
            for key in test_dfs:
                if curr_shares[key] > 0:
                    X = test_dfs[key].iloc[:,:-1]
                    if len(X) == 0:
                        break
                    stock_1_day_pred = one_day_model.predict(X.head(1))
                    stock_1_day_pred = (float(stock_1_day_pred) - X['Close'][0]) / X['Close'][0]
                    
                    if stock_1_day_pred < 0:
                        break
        
        split_time -= day_counter

    return cash_df

In [28]:
cash_df = trading_sim_LRM(stock_objects, drop_list, 280, 5)
cash_df

,Date,Fund Value,Cash,AMZN,AAPL,FB,GOOGL,MSFT,TSLA,RIOT
0,2020-02-07,10000,10000,0,0,0,0,0,0,0
1,2020-02-10,10000,1.11987,0,0,0,0,0,0,6756
2,2020-02-18,10472.9,1.11987,0,0,0,0,0,0,6756
3,2020-02-25,9121.72,535.72,0,0,0,6,0,0,0
4,2020-03-03,8921.8,66.8004,0,0,0,0,0,55,0
5,2020-03-10,7320.53,66.8004,0,0,0,0,0,55,0
6,2020-03-17,4906.91,66.8004,0,0,0,0,0,55,0
7,2020-03-24,5317.1,66.8004,0,0,0,0,0,55,0
8,2020-03-31,5580.55,5580.55,0,0,0,0,0,0,0
9,2020-04-07,5580.55,0.640086,0,0,0,0,0,0,4689
